In [14]:
import requests
import pandas as pd
import feedparser
import time
import selenium
import json
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [6]:
url = 'https://bcs-express.ru/category/torgovye-rekomendacii'
driver = webdriver.Chrome()
driver.get(url)

In [7]:
data = {'title': [], 'href': []}
scroll_pause_time = 2
scroll_height = driver.execute_script("return document.body.scrollHeight")
last_height = 0
button = None

In [8]:
while True:
    driver.execute_script("window.scrollBy(0, 800)")
    time.sleep(scroll_pause_time)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    titles = soup.find_all('a', class_='iKzE')

    for headline in titles:
        data['title'].append(headline)
        href = headline.get('href')
        data['href'].append(href)

    new_scroll_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_scroll_height == scroll_height:
        print('Достигнут конец страницы')
        try:
            button = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-id='button-more']"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", button)
            time.sleep(scroll_pause_time)
            driver.execute_script("arguments[0].click();", button)
            print("Кнопка успешно нажата.")
            time.sleep(scroll_pause_time)
        except:
            print('Заебал листать долбоёб')
            break
        
    scroll_height = new_scroll_height

Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Заебал листать долбоёб


In [23]:
text_dict = {'title': [], 'href': []}

for tit in list(set(data['title'])):
    title = tit.get_text()
    text_dict['title'].append(title)
    text_dict['href'].append(tit.get('href'))

In [28]:
with open('/Users/alexanderknyshov/Desktop/LLM/Data/drafts/result.json', 'w') as fp:
    json.dump(text_dict, fp)

In [34]:
df = pd.read_json('/Users/alexanderknyshov/Desktop/LLM/Data/drafts/result.json')

In [52]:
links = df['hrefs']

In [107]:
data_d = {'description': [], 'publishedAt': []}

In [106]:
driver.get(links[0])
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

d = soup.find_all('chlen', class_='vwgdda')
d

[]

In [108]:
driver = webdriver.Chrome()
counter = 0

for link in links:
    counter += 1
    driver.get(link)
    if counter == 1:
        time.sleep(10)
    else:
        time.sleep(0.2)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    dates = soup.find_all('time', class_='vwGl')
    time.sleep(0.1)
    descriptions = soup.find_all('div', class_='YjHz UBOr RkGZ')
    time.sleep(0.1)

    if descriptions == []:
        data_d['description'].append('void')
    if dates == []:
        data_d['publishedAt'].append('void')

    for description in descriptions:
        desc = description.get_text()
        data_d['description'].append(desc)
        
    for date in dates:
        d = date.get_text()
        data_d['publishedAt'].append(d)
    
    print(f'Описаний записано: {counter}')
    
    time.sleep(0.1)

Описаний записано: 1
Описаний записано: 2
Описаний записано: 3
Описаний записано: 4
Описаний записано: 5
Описаний записано: 6
Описаний записано: 7
Описаний записано: 8
Описаний записано: 9
Описаний записано: 10
Описаний записано: 11
Описаний записано: 12
Описаний записано: 13
Описаний записано: 14
Описаний записано: 15
Описаний записано: 16
Описаний записано: 17
Описаний записано: 18
Описаний записано: 19
Описаний записано: 20
Описаний записано: 21
Описаний записано: 22
Описаний записано: 23
Описаний записано: 24
Описаний записано: 25
Описаний записано: 26
Описаний записано: 27
Описаний записано: 28
Описаний записано: 29
Описаний записано: 30
Описаний записано: 31
Описаний записано: 32
Описаний записано: 33
Описаний записано: 34
Описаний записано: 35
Описаний записано: 36
Описаний записано: 37
Описаний записано: 38
Описаний записано: 39
Описаний записано: 40
Описаний записано: 41
Описаний записано: 42
Описаний записано: 43
Описаний записано: 44
Описаний записано: 45
Описаний записано: 

In [111]:
with open('/Users/alexanderknyshov/Desktop/LLM/Data/drafts/result_descriptions.json', 'w') as fp:
    json.dump(data_d, fp)

In [7]:
df_descriptions = pd.read_json('/Users/alexanderknyshov/Desktop/LLM/Data/drafts/result_descriptions.json')

In [9]:
df_titles = pd.read_json('/Users/alexanderknyshov/Desktop/LLM/Data/drafts/result.json')

In [8]:
df_descriptions

,description,publishedAt
0,На текущий момент бумага торгуется около значе...,11 ноября 2015
1,В данный момент бумаги Сбербанка торгуются око...,31 июля 2015
2,Фьючерс на индекс РТС сегодня изо всех сил пыт...,4 августа 2015
3,Акции МТС сегодня получили дополнительный драй...,6 сентября 2017
4,Настоящий down-тренд\nРоссийский рынок акций у...,19 июня 2017
...,...,...
1630,Фьючерс на индекс РТС продолжает постепенно сн...,29 июля 2016
1631,На этой неделе акции скорректировались вниз по...,14 октября 2015
1632,"В данный момент акции НЛМК торгуются по 112,71...",9 января 2017
1633,На текущий момент бумага торгуется у отметки 4...,7 февраля 2018


In [10]:
df_titles

,titles,hrefs
0,Аэрофлот. Классический трейд в диапазоне,https://bcs-express.ru/novosti-i-analitika/sur...
1,LyondellBasell Industries — мировой лидер в пр...,https://bcs-express.ru/novosti-i-analitika/sbe...
2,"Сбербанк: Появились сигналы к продажам, но пок...",https://bcs-express.ru/novosti-i-analitika/tor...
3,МосБиржа: Покупаем на выходе из канала,https://bcs-express.ru/novosti-i-analitika/mts...
4,ГАЗПРОМ: Отскок от уровня сопротивления,https://bcs-express.ru/novosti-i-analitika/kak...
...,...,...
1630,МосБиржа: Не спешим закрывать позицию,https://bcs-express.ru/novosti-i-analitika/tor...
1631,Русал РДР: Первая цель роста выполнена,https://bcs-express.ru/novosti-i-analitika/luk...
1632,ММК: Лонг приносит неплохой результат,https://bcs-express.ru/novosti-i-analitika/nlm...
1633,"Pinterest. Двойное дно — рискованно, но есть п...",https://bcs-express.ru/novosti-i-analitika/mag...


In [11]:
df_result = pd.concat([df_titles, df_descriptions], axis=1)

In [13]:
df_result.to_json('/Users/alexanderknyshov/Desktop/LLM/Data/datasets/recommendations.json', orient='records', force_ascii=False)

In [18]:
os.remove('/Users/alexanderknyshov/Desktop/LLM/Data/drafts/result.json')
os.remove('/Users/alexanderknyshov/Desktop/LLM/Data/drafts/result_descriptions.json')